In [1]:
import pickle
with open('train.pkl', 'rb') as f:
    x, y = pickle.load(f)

print(x.shape, y.shape)  # (number_of_samples, height, width) (number_of_samples,)

(30000, 1, 150, 150) (30000,)


In [2]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=3)

In [3]:
import numpy as np

# Reshape X_train to the correct shape and duplicate the grayscale channel across 3 channels
X_train_rgb = np.repeat(x, 3, axis=1)

# Ensure the shape is now (24000, 3, 150, 150)
print(X_train_rgb.shape)

# If using TensorFlow, which expects the channel last format, you might need to transpose the axes
x = np.transpose(X_train_rgb, (0, 2, 3, 1))

# Ensure the shape is now (24000, 150, 150, 3) for TensorFlow
print(X_train_rgb.shape)


(30000, 3, 150, 150)
(30000, 3, 150, 150)


In [4]:
x.shape

(30000, 150, 150, 3)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the ImageDataGenerator with only horizontal flip augmentation
from sklearn.model_selection import train_test_split

# Assuming `x` and `y` are your full dataset loaded from .npy files
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

train_datagen = ImageDataGenerator(
    horizontal_flip=True  # Enable horizontal flipping
    # rescale=1./255  # Rescale pixel values
)

# Create a training data generator
train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=32  # Adjust batch size as needed
)

test_datagen = ImageDataGenerator(
     horizontal_flip=True
    # rescale=1./255  # Rescale pixel values
)

# Create a test data generator
test_generator = test_datagen.flow(
    x_test, y_test,
    batch_size=32,  # Adjust batch size as needed
    shuffle=False  # Usually, you don't need to shuffle the test data
)


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers


In [14]:
from tensorflow.keras import layers, models, Model
import tensorflow as tf
import numpy as np

class TinyVGGTF(Model):
    def __init__(self, input_shape, hidden_units, num_classes, use_skip_connections=True, use_batch_norm=True, use_dropout=True):
        super(TinyVGGTF, self).__init__()
        self.use_skip_connections = use_skip_connections

        # Convolutional blocks
        self.conv_block_1 = self._make_conv_block(hidden_units, use_batch_norm, use_dropout, first_block=True, input_shape=input_shape)
        self.conv_block_2 = self._make_conv_block(hidden_units, use_batch_norm, use_dropout)

        # Classifier
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(num_classes, activation='softmax')
        
    def _make_conv_block(self, hidden_units, use_batch_norm, use_dropout, first_block=False, input_shape=None):
        layers_list = []
        if first_block:
            layers_list.append(layers.Conv2D(hidden_units, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=input_shape))
        else:
            layers_list.append(layers.Conv2D(hidden_units, kernel_size=3, strides=1, padding='same', activation='relu'))
        if use_batch_norm:
            layers_list.append(layers.BatchNormalization())
        layers_list.append(layers.Conv2D(hidden_units, kernel_size=3, strides=1, padding='same', activation='relu'))
        if use_batch_norm:
            layers_list.append(layers.BatchNormalization())
        layers_list.append(layers.MaxPooling2D(pool_size=(2, 2)))
        if use_dropout:
            layers_list.append(layers.Dropout(0.25))

        return models.Sequential(layers_list)

    def call(self, inputs):
        x = self.conv_block_1(inputs)
        block_1_output = x
        x = self.conv_block_2(x)

        if self.use_skip_connections:
            # Resize block_1_output to match x's dimensions before adding
            target_shape = x.shape[1:3]
            resized_block_1_output = tf.image.resize(block_1_output, target_shape, method='nearest')
            x = layers.Add()([x, resized_block_1_output])
        
        x = self.flatten(x)
        return self.dense(x)


In [15]:
# Adjust the parameters according to your dataset and requirements
input_shape = (150, 150, 3)  # Input shape of your images
hidden_units = 32  # Number of filters in the convolutional layers
num_classes = 3  # Setting the number of classes to 3 as specified

# Instantiate the model
model = TinyVGGTF(input_shape, hidden_units, num_classes, use_skip_connections=True, use_batch_norm=True, use_dropout=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create a dummy input to build the model
dummy_input = np.random.random((1, *input_shape)).astype(np.float32)

# Pass the dummy input through the model to build it
model(dummy_input)

# Now, you can print the model summary
print(model.summary())


Model: "tiny_vggtf_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_6 (Sequential)   (None, 75, 75, 32)        10400     
                                                                 
 sequential_7 (Sequential)   (1, 37, 37, 32)           18752     
                                                                 
 flatten_3 (Flatten)         multiple                  0         
                                                                 
 dense_3 (Dense)             multiple                  131427    
                                                                 
Total params: 160,579
Trainable params: 160,323
Non-trainable params: 256
_________________________________________________________________
None


In [16]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)


Name: /physical_device:GPU:0   Type: GPU


In [13]:
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Laptop GPU, compute capability 8.6


In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // 32,  # Ensure this matches your batch size
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=test_generator,
    validation_steps=len(x_test) // 32  # Ensure this matches your batch size
)


Epoch 1/10
750/750 [==============================] - 34s 45ms/step - loss: 2.2587 - accuracy: 0.3384 - val_loss: 1.6710 - val_accuracy: 0.3357
Epoch 2/10
750/750 [==============================] - 33s 44ms/step - loss: 1.9945 - accuracy: 0.3352 - val_loss: 2.0928 - val_accuracy: 0.3332
Epoch 3/10
750/750 [==============================] - 36s 49ms/step - loss: 1.5790 - accuracy: 0.3406 - val_loss: 1.4190 - val_accuracy: 0.3319
Epoch 4/10
750/750 [==============================] - 39s 52ms/step - loss: 1.2559 - accuracy: 0.3343 - val_loss: 1.3045 - val_accuracy: 0.3349
Epoch 5/10
750/750 [==============================] - 48s 64ms/step - loss: 1.1358 - accuracy: 0.3414 - val_loss: 1.1164 - val_accuracy: 0.3332
Epoch 6/10
750/750 [==============================] - 54s 72ms/step - loss: 1.1061 - accuracy: 0.3427 - val_loss: 1.1079 - val_accuracy: 0.3344
Epoch 7/10
750/750 [==============================] - 58s 77ms/step - loss: 1.1016 - accuracy: 0.3405 - val_loss: 1.1408 - val_accuracy: